# Creating and modifying Topologies

PyGromosTools offers a wide variaty of tools and functions to assist in the creation and modification of specialized topologies.

In the following notebook we will demonstrate some of the functions and theire usage. These functions and workflows should work with all supported forcfields, but we will use OpenForceField due to the fact that it is independant of make_top and a Gromos++ installation.

In [ ]:
import os
from rdkit.Chem import AllChem

In [ ]:
from pygromos.files.gromos_system.gromos_system import Gromos_System
from pygromos.files.forcefield.openff import openff
from pygromos.files.topology.top import Top

In [ ]:
# setting the work directory
root_dir = os.path.abspath("../example_files/topo_tutorial/")

## Automatic System generation and combining of topologies

In the first part of this notebbok we demonstrate some automatic tools to create and modifie topologies.

In [ ]:
# defining the forcefield we want to use. In case a Gromos system is used make sure to have a valid Gromos++ installation and define the gromos++ path in Gromos_System or add it to you Path.
ff = openff.OpenFF()

In [ ]:
# We define a molecule in RDKit. Note that this molecule could be modified in RDKit or read from sdf, pdb, ... files
mol = AllChem.MolFromSmiles("C1CCCCC1")
mol = AllChem.AddHs(mol)
AllChem.EmbedMolecule(mol)


In [ ]:
# We define our Gromos_system. Note the option auto_convert=True to automatically genarte a valid topology from the molecule defined in mol and the forcefield defined in ff.
grosys = Gromos_System(work_folder=root_dir, system_name="test", forcefield=ff, 
                       rdkitMol=mol, auto_convert=True)

In [ ]:
# The topology created in the last step printed
grosys.top.TITLE

After the automatic creation of this topology for a Cyclohexan molecule we could try to make a liquid topology. Therefore we want to have multiple copies of our topology in one single topology.

In [ ]:
# define a new, empty top
retTop=Top(in_value=None)

With the "+=" operator we can add multiple topologies to a single topology. Or multiply the topology in a for loop. This alows costum topologies for liquids. After the loop we can reasign the variable to our Gromos_System and check if the additional molecules were added.

In [ ]:
for _ in range(10):
    retTop += grosys.top
grosys.top = retTop

In [ ]:
# we added 10 copies of the initial moleucle to retTop and our new topology now contains 10 molecules in the RESNAME block.
grosys.top.RESNAME

In [ ]:
grosys.top.make_ordered()

We can also add different molecules in our system. For example we could add a Toluene to our Cyclohexane system.

In [ ]:
# create a topology for Toluene
tolTop = Gromos_System(work_folder=root_dir, system_name="TOL", in_smiles="c1ccccc1C", forcefield=ff, auto_convert=True).top

In [ ]:
# add the toluene topology to our main topology
grosys.top += tolTop

In [ ]:
tolTop.LJPARAMETERS.content[0].C6

In [ ]:
# check if it was added properly
grosys.top.RESNAME

If many topologies should be added to a single topology the PyGromosTools version of com_top can be used which directly implements multipliers. Instead of for loops.

## Manual topology modifications

In this second part of the notebook we demonstrate some functions to manually tweek topologies.

Like with any othe PyGromosTools file class we have the option to brute force rewrite blocks. However, this often requires advanced knowlede of the block structure. Except for a few simple cases like the TITLE block.

In [ ]:
grosys.top.add_block(blocktitle="TITLE", content=["This is a new Title"])

In [ ]:
grosys.top.make_ordered()

In [ ]:
grosys.top.TITLE

For more complexe blocks PyGromosTools provides many functions to modify topologies. For example we could simply add a new soluteatom to the system. This would be relevant if you want to manually add a ion with very specific Nonbonded values to you system. These tools are vary powrful, but don't do any sanity checks on the values or compatibility to your forcefields.

In [ ]:
# We add a ion named 'XX' with charge +1 and certain LJ-Parameters to the main topology
grosys.top.add_new_atom(PANM="XX", CG=1, C6=1e-3, C12=1e-6, verbose=True)

In [ ]:
# Check for the new Atomtype 'XX'
grosys.top.ATOMTYPENAME

In [ ]:
grosys.top.SOLUTEATOM

In [ ]:
grosys.rebase_files()